In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from tscluster.opttscluster import OptTSCluster
from tscluster.preprocessing.utils import load_data

import warnings
warnings.filterwarnings('ignore')

import piccard as pc
import piccard2 as pc2

# unused
import networkx as nx
import re

In [ ]:
households_data_2021 = gpd.read_file("data_testing/households_data_2021.geojson")
households_data_2016 = gpd.read_file("data_testing/households_data_2016.geojson")
households_data_2011 = gpd.read_file("data_testing/households_data_2011.geojson")

households_data_2021.rename(columns={'v_CA21_434: Occupied private dwellings by structural type of dwelling data': 'occupied_private_dwellings',
                                     'v_CA21_435: Single-detached house': 'single_detached_house',
                                     'v_CA21_440: Apartment in a building that has five or more storeys': 'apt_five_or_more'}, inplace=True)
households_data_2016.rename(columns={'v_CA16_408: Occupied private dwellings by structural type of dwelling data': 'occupied_private_dwellings',
                                     'v_CA16_409: Single-detached house': 'single_detached_house',
                                     'v_CA16_410: Apartment in a building that has five or more storeys': 'apt_five_or_more'}, inplace=True)
households_data_2011.rename(columns={'v_CA11F_199: Total number of occupied private dwellings by structural type of dwelling': 'occupied_private_dwellings',
                                     'v_CA11F_200: Single-detached house': 'single_detached_house',
                                     'v_CA11F_201: Apartment, building that has five or more storeys': 'apt_five_or_more',}, inplace=True)

In [ ]:
census_dfs = [households_data_2011, households_data_2021]
years = ['2011', '2021']

network_table = pc.create_network_table(census_dfs, years, 'GeoUID')
network_table

In [ ]:
from tscluster.tsplot import tsplot

arr, label_dict = pc2.clustering_prep(network_table, 'name', ['occupied_private_dwellings_2011', 'single_detached_house_2011', 'apt_five_or_more_2011', 'occupied_private_dwellings_2021', 'single_detached_house_2021', 'apt_five_or_more_2021'])
fig, ax = tsplot.plot(X=arr, label_dict=label_dict)

In [ ]:
# we will use the elbow method to determine the optimal number of clusters
import setuptools
from sympy import false
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans

model = KMeans(random_state=4,n_init=10)
visualizer = KElbowVisualizer(model, k=(2,10),timings=False, ax=plt.gca())
visualizer.fit(arr)       # Fit data to visualizer
plt.title('Elbow Method for Optimal k using Sum of Square Error')
plt.xlabel('k')
plt.ylabel('Sum of Squared Error')
plt.show()

In [ ]:
G = pc.create_network(census_dfs, years, 'GeoUID', 0.05)
opt_ts = pc2.cluster(network_table, G, 'GeoUID', 4, arr=arr, label_dict=label_dict)

In [ ]:
fig, ax = tsplot.plot(
    X=arr, # the temporal data
    cluster_centers=opt_ts.cluster_centers_, # the cluster centers
    labels=opt_ts.labels_, label_dict=label_dict, # the cluster labels and the label dictionary
    entity_idx=[label_dict['N'].index(i) for i in opt_ts.get_dynamic_entities()[0] if int(i[5:]) % 100 == 0], # show every 100 entities that exhibit cluster label changes
    show_all_entities=False,figsize=(7,16), annot_fontsize = 10,
    xlabel='Year',ylabel='Number'
    ) # show only the entities that exhibit cluster label changes

In [ ]:
import seaborn as sns
from typing import List

def plot_cluster_heatmap(X: np.ndarray,
                         cluster_result: OptTSCluster,
                         n_clusters:int,
                         cluster_labels: List[str]|None = None,
                         label_dict: dict|None = None,
                         figsize: tuple = (10, 7)) -> plt.Figure:
    """
    Plot the cluster heatmap for the temporal data based on the cluster result.

    Parameters
    ----------
    temporal_data : np.ndarray
        The temporal data with shape (n_timesteps, n_samples, n_features).
    cluster_result : OptTSCluster|TSGlobalKmeans|KMeans
        The cluster result from three clustering method of *tscluster*
    n_clusters : int
        The number of clusters in the cluster result.
    cluster_labels : List[str], optional
        The cluster labels, by default None.
        If None, the cluster labels will be 'Cluster 1', 'Cluster 2', ...
    label_dict : dict, optional
        The label dictionary of the temporal data, by default None
        If None, the label dictionary of the cluster result will be used.

    figsize : tuple, optional
        The figure size of the heatmap, by default (10, 7)

    Returns
    -------
    plt.Figure
        The heatmap figure.
    """
    if label_dict is None:
        label_dict = cluster_result.label_dict_

    z_score_matrix = [[[] for _ in range(X.shape[2])]
                      for _ in range(n_clusters)]
    for fsa in range(X.shape[1]):
        current_fsa = cluster_result.labels_[fsa]
        for year in range(X.shape[0]):
            for feature in range(X.shape[2]):
                z_score_matrix[current_fsa[year]][feature].append(X[year,fsa,feature])
    z_score_matrix = np.array(z_score_matrix, dtype=object).T
    z_score_matrix = np.array([[np.mean(z_score_matrix[feature][cluster])
                                for cluster in range(n_clusters)]
                               for feature in range(X.shape[2])])
    result = pd.DataFrame(z_score_matrix, index=label_dict['F'])
    result = result.astype(float)
    if cluster_labels is not None:
        result.columns = cluster_labels
    else:
        result.columns = ['Cluster ' + str(i+1) for i in range(cluster_result.n_clusters)]

    plt.figure(figsize=figsize)
    fig = sns.heatmap(result, cmap='coolwarm', annot=True, fmt=".2f")
    return plt.gcf()

fig = plot_cluster_heatmap(X = arr,
                           cluster_result=opt_ts,
                           n_clusters=3,
                           cluster_labels=['c1','c2','c3'])
                           # assign the cluster labels
plt.title('Heatmap of features by clusters')
plt.xlabel('Clusters')
plt.ylabel('Features')
plt.show()